# Advanced Usage

**IMPORTANT** - Run the getting started notebook first and make sure your environment is set!

- Use [this table](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure%2Cpy#api--feature-support) for the latest information on supported features. 
- Visit the [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python%2Cpy) documentation page for API details and code snippets. 

In this notebook, we'll explore capabilities like structured outputs, developer messages, reasoning effort, and vision support.- 

<br/>

| Characteristic | o1 | o4-mini |
|:--- |:---|:---|
| Developer Messages    | ✅ | ✅ |
| Structured Outputs    | ✅ | ✅ |
| Context Window Input  | 200K | 100K |
| Context Window Output | 200K | 100K |
| Reasoning Effort      | ✅ | ✅ |
| Vision Support        | ✅ | ✅ |
| Chat Completions API  | ✅ | ✅ |
| Responses API         | ✅ |    |
| Functions / Tools     | ✅ | ✅ |
| max_completion_tokens | ✅ |    |
| System messages       | ✅ | ✅ |
| Reasoning summary     | ✅ | |
| Streaming             | ✅ | |
| Model Card | [o4-mini](https://ai.azure.com/explore/models/o4-mini/version/2025-04-16/registry/azure-openai) | [o1](https://ai.azure.com/explore/models/o1/version/2024-12-17/registry/azure-openai)  |
| api_version | 2025-04-01-preview | 2025-03-01-preview |

---

## 1. Developer Messages

Functionally developer messages "role": "developer" are the same as system messages.
However, they are the recommended best practice for setting reasoning goals, persona and context.
Don't specify both system and developer messages - that will confuse the model.
[Learn more](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure%2Cpy#developer-messages)



In [17]:
import os
from openai import AzureOpenAI

developer_message = """
You are a developer assistant. 
Your task is to help me with Python code.
You will receive a prompt that describes the code I want to write.
You return well documented code that is easy to understand.
"""

prompt = """
Write a Python function that takes a list of integers and returns the sum of the even numbers in the list.
"""

reasoning_level = "low"  # Options: low, medium, high


# Set up the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-04-01-preview"
)

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": prompt},
    ],
    max_completion_tokens=5000,
    reasoning_effort=reasoning_level
)

# Print the response
print(response.choices[0].message.content)

Here’s a simple, well‑documented Python function that takes a list of integers and returns the sum of its even numbers:

```python
def sum_even_numbers(numbers):
    """
    Compute the sum of even integers in a list.

    Parameters:
        numbers (list of int): A list of integer values.

    Returns:
        int: The sum of all even numbers in the input list. 
             If there are no even numbers, returns 0.

    Example:
        >>> sum_even_numbers([1, 2, 3, 4, 5, 6])
        12
        # Explanation: 2 + 4 + 6 = 12
    """
    total = 0
    for num in numbers:
        if num % 2 == 0:
            total += num
    return total


# --- Example usage ---
if __name__ == "__main__":
    sample_list = [10, 21, 32, 43, 54, 65]
    result = sum_even_numbers(sample_list)
    print(f"Sum of even numbers in {sample_list} is {result}")
    # Output: Sum of even numbers in [10, 21, 32, 43, 54, 65] is 96
```

Explanation:
- We initialize a running total (`total`) to 0.
- We iterate over 

---

## 2. Structured Inputs

Structured outputs make a model follow a JSON Schema definition that you provide as part of your inference API call. This is in contrast to the older JSON mode feature, which guaranteed valid JSON would be generated, but was unable to ensure strict adherence to the supplied schema. Structured outputs are recommended for function calling, extracting structured data, and building complex multi-step workflows. [Learn more](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python%2Cdotnet-keys&pivots=programming-language-python)

In [18]:
import os
from pydantic import BaseModel
from openai import AzureOpenAI

# Set up the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-04-01-preview"
)

# Define a Pydantic model to represent the event information
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Define a custom parser for the chat completion response
completion = client.beta.chat.completions.parse(
    model="o4-mini", 
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

# Get the parsed event information
event = completion.choices[0].message.parsed
print(event)
#print(completion.model_dump_json(indent=2))

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

## 3. Function Calling 

- Structured Outputs for function calling can be enabled with a single parameter, by supplying strict: true.
- Structured outputs are not supported with parallel function calls. When using structured outputs set parallel_tool_calls to false.
- [Learn more](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python%2Cdotnet-entra-id&pivots=programming-language-python#function-calling-with-structured-outputs)

In [19]:
from enum import Enum
from typing import Union
from pydantic import BaseModel
import openai
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-04-01-preview"
)

class GetDeliveryDate(BaseModel):
    order_id: str

tools = [openai.pydantic_function_tool(GetDeliveryDate)]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order #12345?"}) 

response = client.chat.completions.create(
    model="o4-mini", 
    messages=messages,
    tools=tools
)

print(response.choices[0].message.tool_calls[0].function)
#print(response.model_dump_json(indent=2))

Function(arguments='{"order_id":"12345"}', name='GetDeliveryDate')


---

## 4.a. Vision - Identify
Vision-enabled chat models are large multimodal models (LMM) developed by OpenAI that can analyze images and provide textual responses to questions about them. They incorporate both natural language processing and visual understanding. [Learn more](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/gpt-with-vision?tabs=rest)

Let's try an image from [the Library of Congress](https://www.loc.gov/free-to-use/lighthouses/) and see if it can reason about the location.

<img src="https://www.loc.gov/static/portals/free-to-use/public-domain/lighthouses/lighthouses-2.jpg" alt="NYPL" width="20%">


In [ ]:

from openai import AzureOpenAI
import os

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-04-01-preview"
)

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Can you recognize the location? Tell me about it." 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "https://www.loc.gov/static/portals/free-to-use/public-domain/lighthouses/lighthouses-2.jpg"
                }
            }
        ] } 
    ],
    max_completion_tokens=2000 
)
print(response.choices[0].message.content)

That’s the Cape Hatteras Lighthouse on Hatteras Island in North Carolina’s Outer Banks.  Key points:

• Appearance: 207‑foot‑tall brick tower painted in a distinctive black‑and‑white spiral.  
• History: Completed in 1870 to warn ships off Diamond Shoals, once the “Graveyard of the Atlantic.”  
• Relocation: In 1999, it was moved intact about 2,900 feet inland to protect it from shoreline erosion.  
• National Park Service: Part of Cape Hatteras National Seashore; open for public tours (climbing the 257 steps to the gallery).  
• Landmark status: Tallest brick lighthouse in the U.S. and one of the most‑photographed lighthouses on the Atlantic coast.


## 4.b Vision - Decipher

Let's try an image from [the Library of Congress](https://www.loc.gov/static/portals/free-to-use/public-domain/presidential-papers) and see if it can recognize handwriting and reason about it.

<img src="https://www.loc.gov/static/portals/free-to-use/public-domain/presidential-papers/2.jpg" alt="NYPL" width="20%">


In [3]:
from openai import AzureOpenAI
import os

client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-04-01-preview"
)

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Why is this famous? Summarize it in a short list." 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "https://www.loc.gov/static/portals/free-to-use/public-domain/presidential-papers/2.jpg"
                }
            }
        ] } 
    ],
    max_completion_tokens=2000 
)
print(response.choices[0].message.content)

This is the manuscript of Abraham Lincoln’s Gettysburg Address (November 19, 1863). It’s famous because:  
1. It’s astonishingly brief—just about 270 words—but powerfully distilled.  
2. Lincoln redefined the Civil War as a fight not merely for Union, but for “a new birth of freedom” and true equality (“all men are created equal”).  
3. It consecrated the Gettysburg battlefield as hallowed ground and honored the fallen in a deeply moving way.  
4. Its rhetoric (“government of the people, by the people, for the people”) has become a foundational statement of American democracy.  
5. It transformed public memory of the war and endures as one of the most quoted speeches in U.S. history.


---

## 5. Visual - Local Image


If you want to use a local image, you can use the following Python code to convert it to base64 so it can be passed to the API. Alternative file conversion tools are available online. Learn more about this and [other settings](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/gpt-with-vision?tabs=python#detail-parameter-settings). Let's try an example with this local image.

<img src="./assets/lavacake.webp" alt="Lava Cake" width="50%">

In [21]:
import base64

# Transforming local image to base64
image_path = './assets/lavacake.webp'
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')
image_uri = f"data:image/jpeg;base64,{base64_image}"

# Make a request to the o4-mini model with the image
response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "How many calories are in this ?" 
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": image_uri,
                    "detail" : "high"
                }       
            }
        ] } 
    ],
    max_completion_tokens=2000 
)
print(response.choices[0].message.content)

Here’s a rough breakdown of what I see on the plate and their approximate calories:

• Molten chocolate cake (6–8 cm diameter)  
  – about 300–400 kcal  
• Single scoop vanilla ice cream (≈½ cup)  
  – about 150–180 kcal  
• Berry coulis or compote (a couple of tablespoons)  
  – about 30–60 kcal  
• Sliced strawberries (3–4 berries)  
  – about 10–15 kcal  
• Black or lightly sweetened tea (mug)  
  – negligible to 5 kcal  

Adding these up gives a total in the neighborhood of 500–660 kcal. Of course, actual values will vary with the exact recipe, portion sizes, and any hidden fats or sugars, so consider this a ballpark estimate.


---

## 6. Your Turn - Try It!

Copy one of the above examples and modify it to see how the model responds to different prompts or images.

In [ ]:
# Write your developer message
developer_message = """

"""

# Write your prompt
prompt = """

"""

# Write your reasoning level
reasoning_level = "low"  

# ---------- Run the cell to see result ------
response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": developer_message},
        {"role": "user", "content": prompt},
    ],
    max_completion_tokens=5000,
    reasoning_effort=reasoning_level
)

# Print the response
print(response.choices[0].message.content)
# --------------------------------------------

---